# BERT: finance_sentiment pretrained모델

In [1]:
import os
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import urllib.request
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, \
                            roc_auc_score, confusion_matrix, classification_report, \
                            matthews_corrcoef, cohen_kappa_score, log_loss

In [2]:
pd.set_option('display.max_colwidth',70)

In [3]:
# urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv", filename="./data/finance_data.csv")

In [40]:
MODEL_NAME = "klue/bert-base"
model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, from_pt=True)
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
data = pd.read_csv('./data/finance_data.csv')
print('총 샘플의 수 :',len(data))
data.head()

총 샘플의 수 : 4846


,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to move all production...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로 옮길 계획이 없다고 한다."
1,neutral,Technopolis plans to develop in stages an area of no less than 100...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평방미터 이상의 면적을 단계적으로 개발...
2,negative,The international electronic industry company Elcoteq has laid off...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 해고와는 달리 회사는 사무직 직원 수..."
3,positive,With the new production plant the company would increase its capac...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가시키고 원자재 사용을 개선하여 생산 ...
4,positive,According to the company's updated strategy for the years 2009-201...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 40% 범위의 장기적인 순매출 성장을 ..."


In [5]:
data.sample(30)

,labels,sentence,kor_sentence
658,positive,narrows to EUR2 .8 m 9-mo'09 29 October 2009 - Finnish software an...,2009년 10월 29일 핀란드 소프트웨어 및 하드웨어 개발업체인 EBG1V는 2009년 첫 9개월 동안 순손실이 1년...
3582,neutral,Exel's board of directors will propose a dividend of 0.2 euro $ 0....,엑셀의 이사회는 2007년 4월 19일 연례 총회에서 2006년 주당 0.2유로의 배당금을 제안할 예정이다.
4514,negative,"Rapala VMC Corporation STOCK EXCHANGE RELEASE October 10, 2008 at ...","2008년 10월 10일 오전 11시 45분 라팔라 VMC 코퍼레이션 증권거래소 릴리즈 카우프팅 뱅크 오이지 (""Kap..."
4668,negative,Diluted earnings per share ( EPS ) declined to EUR 0.78 from EUR 1...,희석주당이익(EPS)은 유로 1.76에서 유로 0.78로 감소했다.
522,neutral,"The order consists of capacity expansion, maintenance services and...","이 주문은 용량 확장, 유지보수 서비스, 새로운 충전 기능을 포함하고 있다고 회사측은 말했다."
2076,positive,"According to Olvi's Managing Director Lasse Aho, the company has a...","올비의 상무 이사인 Lasse Aho에 따르면, 그 회사는 핀란드 밖에서 성장을 찾는 것을 목표로 하는 MMX Plus..."
3508,neutral,"After the acquisition, Basware's preliminary pro forma net sales f...","인수 후 바스웨어의 2005년 예비 프로포마 순매출액은 52.6유로, 예비 프로포마 영업이익은 7.1유로다."
4733,negative,"However, its market share shrank to 47.59 per cent from 48 per cen...",그러나 시장점유율은 1년 전의 48%에서 47.59%로 줄어들었다.
2000,positive,`` In the newly formed company YIT Stavo the local contact network...,"`` 주하 코스티아이넨 이이트 기업커뮤니케이션즈비즈니스개발 부사장은 ""새로 설립된 이티스타보에서는 유로스타보콘술트의 현..."
4431,negative,The recent troubles simply make NETeller cheaper.,최근의 문제들은 단순히 넷엘러를 더 저렴하게 만든다.


In [6]:
data = pd.read_csv('./data/finance_data.csv')
print('총 샘플의 수 :',len(data))
data = data.drop(columns=['sentence'])
data.head()

총 샘플의 수 : 4846


,labels,kor_sentence
0,neutral,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로 옮길 계획이 없다고 한다."
1,neutral,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평방미터 이상의 면적을 단계적으로 개발...
2,negative,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 해고와는 달리 회사는 사무직 직원 수..."
3,positive,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가시키고 원자재 사용을 개선하여 생산 ...
4,positive,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 40% 범위의 장기적인 순매출 성장을 ..."


In [7]:
data['labels'] = data['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])
data.head()

,labels,kor_sentence
0,0,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로 옮길 계획이 없다고 한다."
1,0,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평방미터 이상의 면적을 단계적으로 개발...
2,2,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 해고와는 달리 회사는 사무직 직원 수..."
3,1,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가시키고 원자재 사용을 개선하여 생산 ...
4,1,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 40% 범위의 장기적인 순매출 성장을 ..."


In [8]:
print('결측값 여부 :',data.isnull().values.any())

결측값 여부 : False


In [9]:
duplicated = data[data.duplicated()]
duplicated

,labels,kor_sentence
1099,0,이 발표 내용에 대한 책임은 전적으로 발행자에게 있습니다.
1394,0,"핀란드 헬싱키에 본사를 둔 레민카이넨 그룹은 토목 공학, 건축 계약, 기술 건축 서비스, 건축 자재 산업 등 건설 산업..."
1416,0,"이 보고서는 블랙 앤 데커, 피스카스, 피스카스 브랜드, 후스크바르나 아웃도어 프로덕츠, K+S 그룹, 료비 테크놀로지..."
2396,0,알스트롬의 주가는 나스닥 OMX 헬싱키에서 인용되고 있다.
2567,0,SSH 통신 보안 코퍼레이션은 핀란드 헬싱키에 본사를 두고 있다.
2889,0,재정적인 세부사항은 공개되지 않았다.
2890,0,재정적인 세부사항은 공개되지 않았다.
2892,0,금융 조건은 공개되지 않았다.
3050,0,재정적인 세부사항은 제공되지 않았다.
3051,0,재정적인 세부사항은 공개되지 않았다.


In [10]:
# 중복 제거
data.drop_duplicates(subset=['kor_sentence'], inplace=True)
print('총 샘플의 수 :',len(data))

총 샘플의 수 : 4827


In [11]:
print('레이블의 분포')
print(data.groupby('labels').size().reset_index(name='count'))

레이블의 분포
   labels  count
0       0   2861
1       1   1362
2       2    604


In [12]:
X_data = data['kor_sentence']
y_data = data['labels']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0, stratify=y_data)

In [14]:
print('훈련 샘플의 개수 :', len(X_train))
print('테스트 샘플의 개수 :', len(X_test))

훈련 샘플의 개수 : 3861
테스트 샘플의 개수 : 966


In [15]:
X_train

2049                         보험사는 노르데아의 순이익에서 노르데아가 차지하는 비중이 상당할 것으로 예상하고 있다.
129                                                백화점 부서는 매출이 4% 증가했다고 보고했다.
272                                         이 기간의 이익은 1.3 mn에서 5.9 mn으로 증가했다.
3261                      정부는 대신 LMT에 대한 국가 지분을 텔리아소네라의 라떼레콤 지분에 교환할 것을 제안했다.
3937                                        거래 금액은 SEK25m에서 SEK50m 사이로 추정됩니다.
                                        ...                                  
1834                                                    핀란드는 적극적인 배송이 필수적입니다.
2934    헬싱키(Thomson Financial) - M-real은 키로오이로부터 약 13mln유로에 핀란드 키로스코스키에 50...
415     tinyurl 링크는 사용자들이 구글 (NASDAQ : GOOG) 캐시 광고주가 됨으로써 수천 달러를 벌 수 있다고 약...
2489                                            아이온파스의 2대 주주는 벤처캐피털 아우라캐피탈이다.
341     보고기간 중 순매출은 2008-09년 상반기에 실현된 사업인수 영향으로 전년 동기 대비 8% 증가한 64.3m를 기록했...
Name: kor_sentence, Length: 3861, dtype: object

In [16]:
# 훈련 데이터 라벨별 비율
y_train.value_counts(normalize = True)

0    0.592852
1    0.282051
2    0.125097
Name: labels, dtype: float64

In [17]:
# 테스트 데이터 라벨별 비율
y_test.value_counts(normalize = True)

0    0.592133
1    0.282609
2    0.125259
Name: labels, dtype: float64

## BERT 입력용 데이터 포맷으로 변경

In [55]:
MAX_SEQ_LEN = 128

In [55]:
def convert_data(X_data, y_data):
    # BERT 입력으로 들어가는 token, mask, segment, target 
    tokens, masks, segments, targets = [], [], [], []
    
    for X, y in tqdm(zip(X_data, y_data)):
        # token: 입력 문장 토큰화
        token = tokenizer.encode(X, truncation = True, max_length = MAX_SEQ_LEN, padding = 'max_length')
        
        # Mask: 토큰화한 문장 내 패딩이 아닌 경우 1, 패딩인 경우 0으로 초기화
        num_zeros = token.count(0)
        mask = [1] * (MAX_SEQ_LEN - num_zeros) + [0] * num_zeros
        
        # token_type_id: segment인코딩(오직 한 문장이므로 모두 0으로 초기화)
        segment = [0]*MAX_SEQ_LEN

        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        targets.append(y)

    # numpy array로 저장
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

In [65]:
# train 데이터를 Bert의 Input 타입에 맞게 변환
train_x, train_y = convert_text_to_features_data(X_train, y_train,max_seq_len=max_seq_len,tokenizer=tokenizer)

# test 데이터를 Bert의 Input 타입에 맞게 변환
test_x, test_y = convert_text_to_features_data(X_train, y_train,max_seq_len=max_seq_len,tokenizer=tokenizer)

0it [00:00, ?it/s]C:\Users\bitcamp\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
3861it [00:01, 3450.31it/s]
3861it [00:01, 3447.23it/s]


In [66]:
train_x[0].max()

27167

In [67]:
train_x[1].max()

1

In [68]:
train_x[2].max()

0

In [69]:
y_train

2049    1
129     1
272     1
3261    0
3937    0
       ..
1834    0
2934    1
415     2
2489    0
341     1
Name: labels, Length: 3861, dtype: int64

In [70]:
y_test

2347    0
2963    0
4423    2
4735    2
2487    0
       ..
3907    0
957     1
275     1
2448    0
2440    0
Name: labels, Length: 966, dtype: int64

In [62]:
# token, mask, segment 입력 정의
token_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_word_ids')
mask_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_masks')
segment_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_segment')


## BERT출력 정의

In [63]:
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])
bert_output = bert_outputs[0]

## 감정분류모델 컴파일

In [64]:
DROPOUT_RATE = 0.5
NUM_CLASS = 3
dropout = tf.keras.layers.Dropout(DROPOUT_RATE)(bert_output)
# Multi-class classification 문제이므로 activation function은 softmax로 설정
sentiment_layer = tf.keras.layers.Dense(NUM_CLASS, activation='softmax', kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev=0.02))(dropout)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_layer)


In [65]:
# 옵티마이저 Rectified Adam 하이퍼파리미터 조정

OPTIMIZER_NAME = 'RAdam'
LEARNING_RATE = 5e-5
TOTAL_STEPS = 10000
MIN_LR = 1e-5
WARMUP_PROPORTION = 0.1
EPSILON = 1e-8
CLIPNORM = 1.0
optimizer = tfa.optimizers.RectifiedAdam(learning_rate = LEARNING_RATE,
                                          total_steps = TOTAL_STEPS, 
                                          warmup_proportion = WARMUP_PROPORTION, 
                                          min_lr = MIN_LR, 
                                          epsilon = EPSILON,
                                          clipnorm = CLIPNORM)

In [66]:
# 감정분류 모델 컴파일
sentiment_model.compile(optimizer = optimizer, 
                        loss = tf.keras.losses.SparseCategoricalCrossentropy(), 
                        metrics = ['accuracy'])

In [67]:
MIN_DELTA = 1e-3
PATIENCE = 5

early_stopping = EarlyStopping(
    monitor = "val_accuracy", 
    min_delta = MIN_DELTA,
    patience = PATIENCE)

In [68]:
# 최고 성능의 모델 파일을 저장할 이름과 경로 설정
BEST_MODEL_NAME = './data/model/best_model.h5'
model_checkpoint = ModelCheckpoint(
    filepath = BEST_MODEL_NAME,
    monitor = "val_loss",
    mode = "min",
    save_best_only = True, # 성능 향상 시에만 모델 저장
    verbose = 1
)

In [69]:
callbacks = [early_stopping, model_checkpoint]

In [73]:
EPOCHS = 100
BATCH_SZIE = 32

sentiment_model.fit(train_x, train_y, 
                    epochs = EPOCHS, 
                    shuffle = True, 
                    batch_size = BATCH_SZIE, 
                    validation_data = (test_x, test_y),
                    callbacks = callbacks
                    )

In [39]:
test_x[0].max()

31474

In [ ]:
# 최고 성능의 모델 불러오기
sentiment_model_best = tf.keras.models.load_model(BEST_MODEL_NAME,
                                                  custom_objects={'TFBertForSequenceClassification': TFBertForSequenceClassification})

In [ ]:
# 모델이 예측한 라벨 도출
predicted_value = sentiment_model_best.predict(test_x)
predicted_label = np.argmax(predicted_value, axis = 1)

In [ ]:
# Classification Report 저장
CL_REPORT_FILE = "./data/metric/cl_report.csv"

cl_report = classification_report(test_y, predicted_label, output_dict = True)
cl_report_df = pd.DataFrame(cl_report).transpose()
cl_report_df = cl_report_df.round(3)
cl_report_df.to_csv(CL_REPORT_FILE)
print(cl_report_df)

# 모델에 넣어줄 텍스트 데이터 전처리 함수:  text_data()

## 저장된 모델 불러오기

In [12]:
# 최고 성능의 모델 불러오기
BEST_MODEL_NAME = './data/model/best_model.h5'
sentiment_model_best = tf.keras.models.load_model(BEST_MODEL_NAME,
                                                  custom_objects={'TFBertForSequenceClassification': TFBertForSequenceClassification})

In [13]:
MODEL_NAME = "klue/bert-base"
model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, from_pt=True)
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
MAX_SEQ_LEN = 128
def text_data(X_data):
    # BERT 입력으로 들어가는 token, mask, segment, target 저장용 리스트
    input_ids, attention_masks, token_type_ids, targets = [], [], [], []
    
    for X in tqdm(X_data):
        # input_id: 워드 임베딩을 위한 문장의 정수 인코딩
        input_id = tokenizer.encode(X, truncation = True, padding = 'max_length', max_length = MAX_SEQ_LEN)
        
        # attention_mask: 실제 단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스
        num_zeros = token.count(0)
        attention_masks = [1] * (MAX_SEQ_LEN - num_zeros) + [0] * num_zeros
        
        # token_type_id: segment인코딩(오직 한 문장이므로 모두 0으로 초기화)
        token_type_id = [0]*MAX_SEQ_LEN

        input_ids.append(token)
        masks.append(mask)
        segments.append(token_type_id)


    # numpy array로 저장
    input_ids = np.array(input_id)
    attention_masks = np.array(attention_mask)
    segments = np.array(segments)


    return [input_ids, attention_masks, token_type_ids]

In [15]:
df_n = pd.read_csv('./data/개별기업뉴스_전처리/카카오.csv',index_col=0)
df_n.head()

,날짜,뉴스제목,뉴스요약
7111,2016-01-07,카카오·스타벅스·카페베네 모바일상품권 갑질 횡포!,"상품권 유효기간이 너무 짧습니다 장창엽 / 세종시 한솔동 ""선물을 받은 적이 있었는데 기간이 너무 짧아서 놓쳐서 못받고..."
5979,2016-01-11,"멜론 품은 카카오, ""글로벌 진출 발판 마련""","카카오가 멜론을 서비스하는 국내 1위 음악콘텐츠 회사인 로엔을 인수합니다. 무서운 기세로 사업을 확장하는 카카오, 이제..."
5978,2016-01-11,"카카오-로엔, 직접 빅딜…高인수가 논란",<앵커> 지난해부터 M&A로 몸집 불리기에 나섰던 카카오가 새해부터 공격적인 행보를 이어갔습니다. 카카오 사상 최대 규...
5977,2016-01-11,"카카오, 1조 8천억 원에 '음원 시장 1위' 멜론 인수","유명한 카카오가 음원 사이트 멜론을 인수했습니다. 모바일 메신저 1위와 음원 시장 1위 업체의 만남, 콘텐츠 시장에 큰..."
5976,2016-01-11,"카카오, '멜론' 운영사 1조9천억원에 인수","카카오, '멜론' 운영사 1조9천억원에 인수 'IT 공룡' 카카오가 국내 1위 음악 콘텐츠업체 로엔엔터테인먼트를 1조8..."


In [16]:
news_data = df_n['뉴스제목']
len(news_data)

10996

In [17]:
news_x = text_data(news_data)
news_x

100%|██████████████████████████████████████████████████████████████████████████| 10996/10996 [00:02<00:00, 5288.66it/s]


[array([[    2,  7270,   100, ...,     0,     0,     0],
        [    2, 19632,  1873, ...,     0,     0,     0],
        [    2,  7270,    17, ...,     0,     0,     0],
        ...,
        [    2,  7270,  2882, ...,     0,     0,     0],
        [    2,  7270, 13960, ...,     0,     0,     0],
        [    2,     6,  3769, ...,     0,     0,     0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

## 예측 결과

In [18]:
predicted_value = sentiment_model_best.predict(news_x)
predicted_label = np.argmax(predicted_value, axis = 1)
predicted_label

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [19]:
pos_idx = np.where(predicted_label==1)
neg_idx = np.where(predicted_label==2)
neu_idx = np.where(predicted_label==0)

In [20]:
print('긍정개수 :', sum(predicted_label ==1))
print('부정개수 :', sum(predicted_label ==2))
print('중립개수 :', sum(predicted_label ==0))

긍정개수 : 2002
부정개수 : 768
중립개수 : 8226


### 긍정

In [32]:
df_n.iloc[pos_idx].sample(10)

,날짜,뉴스제목,뉴스요약
10365,2021-03-30,"네이버보다 못 오른 카카오, 5대1 액면분할로 주가 부양 가능할까?",카카오(035720)의 5대1 액면분할안이 정기주주총회를 통과하면서 소액주주들에게 문턱이 더 낮아졌다. 올... 29일...
1602,2022-05-27,삼성 금융앱 ‘모니모’ 4월 사용자 수 74만 증가…카카오페이 66만·토스 63...,"카카오페이 사용자 수는 66만명 늘었으며, 토스는 63만명 증가했다. 또한 금융앱 중에서 가장 많은 신규... 순위에서..."
1941,2022-05-04,"카카오, 1분기 영업익 1587억원…광고 줄고 콘텐츠 성장세(상보)",2022년 1분기 카카오 실적 요약[이데일리 이대호 기자] 카카오(035720)가 한국국제회계기준 2022년 1분기 연...
12454,2022-04-27,"“폰타나로 시작하는 든든한 하루!” 폰타나, 카카오 쇼핑라이브 첫 방송 진행",정통 서양식 브랜드 폰타나가 5월 4일 오전 8시 30분부터 ‘폰타나로 시작하는 든든한 하루’ 카카오... 폰타나 관계...
8408,2022-05-03,"카카오게임즈, 1분기 매출 2663억원…전년比 105% ↑",카카오게임즈가 모바일 게임 '오딘'의 인기로 올해 1분기 매출이 전년 대비 2배 증가했다. 카카오게임즈는 2022년 1...
10171,2021-05-07,최고 실적 또 갈아치운 카카오… 모빌리티·페이 ‘일등 공신’,카카오가 또다시 역대 최고 실적을 갈아치웠다. 모빌리티(운송 사업)와 페이(간편결제)를 비롯한 신사업이 90%에 육박하...
3389,2022-05-10,"서울대병원-카카오헬스케어, 정밀의료 지식은행 구축 위한 업무협약 체결","김정아 기자 | 서울대병원과 카카오헬스케어가 데이터의 시각화, 요약, 통계 및 분석 자동화가 가능한 차세대 정밀의료 지..."
1967,2022-05-18,더 똑똑해진 카카오 '톡비서 죠르디'…카톡 메시지도 예약 해준다,죠르디 /사진=카카오 제공 카카오톡에 메시지 예약 전송 기능이 생겼다. 생일 축하나 일정 등 메시지를 미리 저장한 날짜...
16975,2022-05-04,"[리포트 브리핑]카카오게임즈, '신작 없이도 선방한 1분기' 목표가 110,000원...",로보뉴스 = 미래에셋증권에서 04일 카카오게임즈(293490)에 대해 '신작 없이도 선방한 1분기'라며... ◆ 카카오...
17801,2020-08-06,"[잠정실적]카카오 2Q 실적, 영업이익 978억원... 전년동기 대비 142% 증가 (연...",로보뉴스 = 카카오의 영업이익률이 전년동기 대비하여 4.5%p 증가하였다. 19년 06월 5.5%이던 영업이익률이......


In [31]:
df_n.iloc[pos_idx].sort_values(by='날짜',ascending=False)

,날짜,뉴스제목,뉴스요약
198,2022-05-30,"[기획] 카카오, ESG 경영 강화…이미지 쇄신 팔 걷어",카카오는 16일 두번째 ESG 보고서 '2021 카카오의 약속과 책임'을 발간했다. 사진=카카오 제공 카카오가 ESG(...
294,2022-05-30,"카카오페이 대출비교 홍보영상 500만뷰 돌파... ""제1금융권 제휴 최다""",카카오페이 '내 대출 한도' 서비스 이미지[사진=카카오페이] 카카오페이의 대출 비교 서비스 영상의 조회수가 500만뷰를...
73,2022-05-30,"""포용적 금융지원 확대""...카카오뱅크·캠코, 연체채권 정기 인수계약 체결",협력사업 지속 확대 예정 카카오뱅크는 취약·연체 채무자의 주거안정과 경제적 재기지원을 위해 캠코... 카카오뱅크는 주...
8,2022-05-30,[특징주] 저점 찍었나…네이버 4%·카카오 2%대 반등,국내 증시 대표 플랫폼주인 네이버와 카카오 주가가 30일 모처럼 큰 폭으로 반등했다. 이날 유가증권시장에서 네이버는 전...
26,2022-05-30,"카카오뱅크, 캠코와 취약 채무자 주거안정 높인다",카카오뱅크는 취약·연체 채무자의 주거안정과 경제적 재기지원을 위해 한국자산관리공사(캠코)와... 카카오뱅크는 주택담보대...
...,...,...,...
5953,2016-03-08,"카카오 드라이버, 카카오 택시 성공 잇는다",<앵커> 카카오가 대리기사 친화 정책을 내세우며 대리운전 서비스인 `카카오 드라이버` 기사 모집에 나섰습니다. 8만 7...
5963,2016-02-05,카카오 VS 네이버…어디가 더 매력적?,■ 경제와이드 이슈& - 김민수 메리츠종금증권 여의도금융센터 부장 오늘 카카오의 실적 발표가 예정돼 있다. 이미 네이버...
5962,2016-02-05,실적부진 카카오…올해 성장성 입증할까,■ 경제와이드 '백브리핑 시시각각' <앵커> 모바일 플랫폼 기업 카카오가 지난해 경영실적을 발표했습니다.... 윤 기자...
5969,2016-01-12,"[그건 이렇습니다] 카카오, 로엔 인수…신의 한 수? 무리수?",■ 경제와이드 '백브리핑 시시각각' <앵커> 카카오가 국내 음원 시장 1위 사업자 로엔엔터테인먼트를 1조8700억 원이...


### 부정

In [36]:
df_n.iloc[neg_idx].sample(10)

,날짜,뉴스제목,뉴스요약
5196,2021-09-09,[오늘의 경제지표] 네이버·카카오 '이틀째 동반 하락'...여파는?,금융당국의 핀테크 규제 움직임에 네이버와 카카오 등 대형 플랫폼 주들의 주가가 이틀째 무너지며 지수 하락을 주도했습니다...
3317,2022-05-04,"카카오, 1분기 영업익 1587억원... 시장 전망치 밑돌아",카카오 로고. 제공: 카카오 카카오가 올해 1분기 계절적 비수기의 영향과 거시경제 불확실성으로 인해 시장 전망치를 밑도...
4639,2021-03-26,"""부정 배차 없다""는 카카오T…기사들은 ""차별 느낀다""",카카오T는 차량 호출 시장의 80%를 차지하고 있습니다. 그런데 여러 유료 서비스를 만들면서 택시기사들... [기자] ...
5384,2018-12-14,[뉴스초점] 카풀서비스 연기에도 카카오-택시업계 갈등 지속,카풀서비스 연기에도 카카오-택시업계 갈등 지속 카카오모빌리티가 17일 시작하겠다던 카풀 정식서비스를 잠정 연기한다고 발...
4126,2022-01-19,카카오 다음 합병 과정 탈세 의혹‥코스피 코스닥 하락,시가 총액 상위 종목 살펴보면 내리는 종목이 더 많은 가운데 김범수 카카오의장과 김범수 의장 소유의케이큐브홀딩스가 카카...
16913,2021-12-07,"07일, 외국인 거래소에서 SK스퀘어(-0.61%), 카카오(-0.41%) 등 순매도","외국인 투자자는 07일 거래소에서 SK스퀘어, 카카오, 코리아써키트 등을 중점적으로 매도한 것으로 나타났다. 외국인 투..."
10621,2022-01-12,카카오 ‘먹튀’ 논란… 보험업 진출에도 악재로,내줘 카카오페이 경영진의 ‘먹튀’ 논란 여파로 리더십이 흔들리면서 향후 사업에 악재가 쌓이고 있다. 카카오페이가 진출을...
9824,2021-07-11,"카카오, '클럽하우스' 잡긴 했지만···대중 관심은 시들",카카오(035720)가 선보인 소셜 오디오 플랫폼 ‘음(mm)’ 국내 이용자가 ‘원조’ 클럽하우스를 넘어섰다. 초대장이...
2603,2022-05-02,"카카오페이, 1분기 영업손실 11억원…주가는 장중 신저가",핀테크기업 카카오페이가 올해 1분기 영업손실 11억원(연결 기준)을 기록해 전년 동기 대비 적자로 전환했다고 2일 공시...
9506,2021-09-08,정부·여당發 규제 협공에…네이버·카카오 주가 폭락,"국내 플랫폼기업 전방위 타격 카카오페이 ""서비스 보완할것"" 정치권과 금융당국이 핀테크 산업을 향한 규제 도입을 공언하면..."


### 중립

In [37]:
df_n.iloc[neu_idx].sample(10)

,날짜,뉴스제목,뉴스요약
9817,2021-07-22,"공정위, 카카오T·딜카 기업 결합 승인",DH ‘요기요’ 매각기한 연장 공정거래위원회가 택시호출 서비스인 ‘카카오T’를 운영하는 카카오모빌리티가 현대캐피탈의 온...
8974,2021-12-08,CJ는 미국 카카오는 유럽으로…K엔터 영토 무한확장 나섰다,콘텐츠산업 해외 공략 속도 카카오도 佛 웹툰시장 진출 CJ ENM과 카카오엔터테인먼트 등 국내 콘텐츠 제작사들이 세계 ...
3872,2022-05-04,"스카이랩스, 카카오헬스케어와 MOU…""모바일 만성질환 관리 시스템 구축""","헬스케어 스타트업 스카이랩스가 카카오헬스케어와 모바일 만성질환 관리 시스템 구축을 위해 업무협약... 있는데, 카카오헬..."
9820,2021-07-22,"카카오T, 쏘카와의 한판승부 나서다",공정거래위원회는 22일 카카오모빌리티가 현대캐피탈 ‘딜카’을 인수하는 기업결합 건에 대해 경쟁제한 우려가 적다고 판단해...
1705,2022-05-17,"카카오엔터, '브로커' '헌트' 등 다양한 작품들로 세계 시장 정조준","카카오엔터테인먼트는 직접 기획, 제작한 작품들은 물론 산하 자회사들과 함께한 작품들이 전세계... 공개되는 카카오TV ..."
12285,2022-05-09,"카카오브레인, 페이스 스와핑 기술 개발... '메타버스 시대 이끈다'",뉴데일리 김동준 기자 = 카카오브레인이 아이덴티티 임베딩(Identity Embedding) 기능이 향상된 새로운......
8876,2022-01-10,"[기자의 눈/지민구]카카오, 커진 덩치만큼 성숙한 책임 보여야","“세상에서 가장 비싼 자산이 신뢰인데, 시장과 임직원의 신뢰를 모두 잃어버렸습니다.” 직장인 익명 커뮤니티 블라인드 등..."
10920,2020-11-11,"""가보지 않은 길"" 김범수가 그리는 카카오 시즌2","[편집자주] 카카오가 폭풍 성장했다. 분기 매출만 벌써 1조원을 돌파했다. 스타트업에서 출발해 현재는 임직원 1만명, ..."
11232,2021-11-16,"카카오, 한글 특화 인공지능 공개…네이버와 '토종 AI 경쟁' 불붙었다",카카오가 ‘초(超)거대 인공지능(AI)’ 구축을 본격화한다. 한국어 특화 AI 언어 모델 ‘KoGPT’를 처음... 한...
5994,2016-06-10,대기업 딱지 두 달 만에 도로 뗀 카카오·셀트리온,"카카오와 셀트리온 등 37개 기업이 대기업 굴레에서 벗어나게 됐는데, 투자 확대로 이어질지 주목됩니다. 김경기 기자가 ..."


# 합치기

In [3]:
# 최고 성능의 모델 불러오기
BEST_MODEL_NAME = './data/model/best_model.h5'
sentiment_model_best = tf.keras.models.load_model(BEST_MODEL_NAME,
                                                  custom_objects={'TFBertForSequenceClassification': TFBertForSequenceClassification})
MODEL_NAME = "klue/bert-base"
model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, from_pt=True)
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
MAX_SEQ_LEN = 64
def text_data(X_data):
    # BERT 입력으로 들어가는 token, mask, segment, target 저장용 리스트
    tokens, masks, segments, targets = [], [], [], []
    
    for X in tqdm(X_data):
        # token: 입력 문장 토큰화
        token = tokenizer.encode(X, truncation = True, padding = 'max_length', max_length = MAX_SEQ_LEN)
        
        # Mask: 토큰화한 문장 내 패딩이 아닌 경우 1, 패딩인 경우 0으로 초기화
        num_zeros = token.count(0)
        mask = [1] * (MAX_SEQ_LEN - num_zeros) + [0] * num_zeros
        
        # segment: 문장 전후관계 구분: 오직 한 문장이므로 모두 0으로 초기화
        segment = [0]*MAX_SEQ_LEN

        tokens.append(token)
        masks.append(mask)
        segments.append(segment)


    # numpy array로 저장
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)


    return [tokens, masks, segments]

In [6]:
df_n = pd.read_csv('./data/개별기업뉴스_전처리/AJ네트웍스.csv',index_col=0)
news_data = df_n['뉴스제목']
news_x = text_data(news_data)
predicted_value = sentiment_model_best.predict(news_x)
predicted_label = np.argmax(predicted_value, axis = 1)
predicted_label = pd.Series(predicted_label).apply(lambda x: -1 if x== 2 else x)
predicted_label

100%|██████████████████████████████████████████████████████████████████████████████| 907/907 [00:00<00:00, 5301.94it/s]


0      0
1      0
2      0
3      0
4      0
      ..
902    1
903    0
904    1
905    0
906    1
Length: 907, dtype: int64

In [31]:
path = './data/개별기업뉴스_전처리/'
file_list = os.listdir(path)
cnt = 0
for file in file_list[:]:
    cnt+=1
    corp_name = file[:-4]
    print('인덱스 :',cnt, '회사명 :',corp_name)
    file_path = os.path.join(path,file)
    df_n = pd.read_csv(file_path,index_col=0)
    df_n.reset_index(drop=True, inplace=True)
    
    news_data = df_n['뉴스제목']
    news_x = text_data(news_data)
    
    predicted_value = sentiment_model_best.predict(news_x)
    predicted_label = np.argmax(predicted_value, axis = 1)
    predicted_label = pd.Series(predicted_label).apply(lambda x: -1 if x== 2 else x )
    df_n['감성점수'] = predicted_label
    
    df_n.to_csv('./data/데이터_뉴스+감성점수/{}.csv'.format(corp_name))


인덱스 : 1 회사명 : AJ네트웍스


100%|██████████████████████████████████████████████████████████████████████████████| 907/907 [00:00<00:00, 4993.71it/s]


인덱스 : 2 회사명 : AK홀딩스


100%|██████████████████████████████████████████████████████████████████████████████| 837/837 [00:00<00:00, 5173.85it/s]


인덱스 : 3 회사명 : BGF


...

100%|██████████████████████████████████████████████████████████████████████████████| 878/878 [00:00<00:00, 5075.06it/s]


인덱스 : 820 회사명 : 흥국화재


100%|████████████████████████████████████████████████████████████████████████████| 2027/2027 [00:00<00:00, 5388.71it/s]


인덱스 : 821 회사명 : 흥아해운


100%|████████████████████████████████████████████████████████████████████████████| 1113/1113 [00:00<00:00, 5335.42it/s]


In [25]:
predicted_label

0      0
1      0
2      0
3      0
4      0
      ..
902    1
903    0
904    1
905    0
906    1
Length: 907, dtype: int64

In [26]:
df_n['감성점수']

0      0
1      0
2      0
3      0
4      0
      ..
902    1
903    0
904    1
905    0
906    1
Name: 감성점수, Length: 907, dtype: int64

In [27]:
temp = pd.read_csv('./data/데이터_뉴스+감성점수/AJ네트웍스.csv',index_col=0)
temp.head()

,날짜,뉴스제목,뉴스요약,감성점수
0,2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'",AJ네트웍스가 SK브로드밴드를 통해 진행중인 '스마트렌탈'의 위약금 규정 및 취급상...,0
1,2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작","국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,...",0
2,2016-01-06,"AJ네트웍스, 스마트폰, 태블릿PC 등 Mobile Device 렌탈 사업 시작","국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,...",0
3,2016-01-13,"AJ네트웍스, 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭",AJ네트웍스(사장 반채운)가 파렛트와 고소장비 렌탈 부문의 신규 서비스 브랜드를 런...,0
4,2016-01-22,대상·AJ네트웍스 등 비우량채도 잇달아 발행 성공,신용등급이 BBB+인 AJ네트웍스도 회사채 발행을 위해 지난 20일 투자자들을 대상...,0


In [28]:
temp.groupby('날짜')['감성점수'].sum()

날짜
2016-01-04    0
2016-01-05    0
2016-01-06    0
2016-01-13    0
2016-01-22    0
             ..
2022-05-14    0
2022-05-15    1
2022-05-16    1
2022-05-23    1
2022-05-25    2
Name: 감성점수, Length: 558, dtype: int64

In [ ]:
max_seq_len = 128
def convert_text_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드 임베딩을 위한 문장의 정수 인코딩
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)

        # attention_mask는 실제 단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

        # token_type_id은 segment 인코딩(한 문장이므로 모두 0)
        token_type_id = [0] * max_seq_len

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)
        
    # numpy array 형태로 저장
    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

#  Gitub: github.com/ukairia777/finance_sentiment_corpus/blob/main/BERT_sentiment_analysis_kor.ipynb

In [18]:
max_seq_len = 64

In [19]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [20]:
max_seq_len = 64
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드 임베딩을 위한 문장의 정수 인코딩
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)

        # attention_mask는 실제 단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

        # token_type_id은 segment 인코딩(한 문장이므로 모두 0)
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [21]:
train_X, train_y = convert_examples_to_features(X_train, y_train, max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|                                                                                         | 0/3861 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\bitcamp\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for

In [22]:
test_X, test_y = convert_examples_to_features(X_test, y_test, max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|                                                                                          | 0/966 [00:00<?, ?it/s]C:\Users\bitcamp\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████| 966/966 [00:00<00:00, 3449.96it/s]


In [23]:
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=3, from_pt=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
early_stopping = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001,
    patience=2)

with tf.device('/GPU:0'):
    model.fit(
        train_X, train_y, epochs=2, batch_size=32, validation_split=0.2,
        callbacks = [early_stopping]
    )

Epoch 1/2


C:\Users\bitcamp\anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py:376: FutureWarning: The old compute_loss method is deprecated as it conflicts with the Keras compute_loss method added in TF 2.8. If you want the original HF compute_loss, please call hf_compute_loss() instead. From TF versions >= 2.8, or Transformers versions >= 5, calling compute_loss() will get the Keras method instead.
  return py_builtins.overload_of(f)(*args)


97/97 [==============================] - 47s 347ms/step - loss: 0.5478 - accuracy: 0.7691 - val_loss: 0.5373 - val_accuracy: 0.7943
Epoch 2/2
97/97 [==============================] - 32s 326ms/step - loss: 0.2771 - accuracy: 0.8944 - val_loss: 0.4599 - val_accuracy: 0.8111


In [26]:
model.evaluate(test_X, test_y, batch_size=1024)

1/1 [==============================] - 2s 2s/step - loss: 0.4109 - accuracy: 0.8416


[0.41085386276245117, 0.8416149020195007]

In [ ]:
# 최고 성능의 모델 파일을 저장할 이름과 경로 설정
BEST_MODEL_NAME = './data/model/best_model2.h5'
model_checkpoint = ModelCheckpoint(
    filepath = BEST_MODEL_NAME,
    monitor = "val_loss",
    mode = "min",
    save_best_only = True, # 성능 향상 시에만 모델 저장
    verbose = 1
)

In [ ]:
callbacks = [early_stopping, model_checkpoint]

EPOCHS = 100
BATCH_SZIE = 32

sentiment_model.fit(train_x, train_y, 
                    epochs = EPOCHS, 
                    shuffle = True, 
                    batch_size = BATCH_SZIE, 
                    validation_data = (test_x, test_y),
                    callbacks = callbacks
                    )